<a href="https://colab.research.google.com/github/rahulkhattri0/Fake-news-detection/blob/main/FakeNews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

In [2]:
# URL to the dataset
dataset_url = "https://www.kaggle.com/c/fake-news/data"

In [4]:
# Download the dataset manually from Kaggle and save it as "train.csv" in your working directory.
# Load the dataset using pandas
df = pd.read_csv("train.csv")

In [5]:
# Display the first few rows of the dataset
print(df.head())

   id                                              title              author  \
0   0  House Dem Aide: We Didn’t Even See Comey’s Let...       Darrell Lucus   
1   1  FLYNN: Hillary Clinton, Big Woman on Campus - ...     Daniel J. Flynn   
2   2                  Why the Truth Might Get You Fired  Consortiumnews.com   
3   3  15 Civilians Killed In Single US Airstrike Hav...     Jessica Purkiss   
4   4  Iranian woman jailed for fictional unpublished...      Howard Portnoy   

                                                text  label  
0  House Dem Aide: We Didn’t Even See Comey’s Let...      1  
1  Ever get the feeling your life circles the rou...      0  
2  Why the Truth Might Get You Fired October 29, ...      1  
3  Videos 15 Civilians Killed In Single US Airstr...      1  
4  Print \nAn Iranian woman has been sentenced to...      1  


In [6]:
# data preprocessing now
import pandas as pd
import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

In [7]:
# Define functions for preprocessing
def clean_text(text):
    if isinstance(text, str):
        # Remove HTML tags
        text = re.sub(r'<.*?>', '', text)
        # Remove special characters, numbers, and symbols
        text = re.sub(r'[^\w\s]', '', text)
        # Remove extra whitespace
        text = re.sub(r'\s+', ' ', text).strip()
    return text

def preprocess_text(text):
    if isinstance(text, str):
        # Tokenization (split text into words)
        words = text.split()
        # Lowercasing
        words = [word.lower() for word in words]
        # Stopword removal
        stop_words = set(stopwords.words('english'))
        words = [word for word in words if word not in stop_words]
        # Lemmatization
        lemmatizer = WordNetLemmatizer()
        words = [lemmatizer.lemmatize(word) for word in words]
        # Join the words back into a single string
        return ' '.join(words)
    else:
        return text

In [8]:
# Download the "stopwords" and "wordnet" dataset
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [9]:
# Apply text cleaning and preprocessing to the 'text' column
df['text'] = df['text'].apply(clean_text)
df['text'] = df['text'].apply(preprocess_text)

In [10]:
# Handling missing data (remove rows with missing text)
df.dropna(subset=['text'], inplace=True)

In [11]:
# Display the preprocessed text
print(df['text'].head())

0    house dem aide didnt even see comeys letter ja...
1    ever get feeling life circle roundabout rather...
2    truth might get fired october 29 2016 tension ...
3    video 15 civilian killed single u airstrike id...
4    print iranian woman sentenced six year prison ...
Name: text, dtype: object


In [12]:
# Install the necessary libraries
!pip install transformers
!pip install langdetect

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 56.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 32.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 108.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 82.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 13.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993224 sha256=62e8b035d17cc48c67ae605937e5cff6a214f079e6b502c287361f4578718ead
  Stored in directory: /root/.cache/pip/wheels/95/03/7d/59ea870c70ce4e5a370638b5462a7711ab78fba2f655d05106
Successfully built langdetect


In [13]:
# Model Selection and Training with BERT
import numpy as np
import torch
import torch.nn as nn
from transformers import BertTokenizer, BertForSequenceClassification, AdamW, get_linear_schedule_with_warmup
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from langdetect import detect  # Import the language detection library

In [15]:
# Define supported languages (add or remove languages as needed)
supported_languages = ['en', 'fr', 'es', 'de']

# Function to identify the language of a text
def detect_language(text):
    try:
        return detect(text)
    except:
        return "unknown"

# Identify and filter articles by language
df['language'] = df['text'].apply(detect_language)
df_filtered = df[df['language'].isin(supported_languages)].copy()

In [18]:
# Load BERT tokenizer and model for multilingual support
tokenizer = BertTokenizer.from_pretrained("bert-base-multilingual-cased")
model = BertForSequenceClassification.from_pretrained("bert-base-multilingual-cased", num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [19]:
# Tokenize and prepare data for BERT
X_train, X_test, y_train, y_test = train_test_split(
    df_filtered['text'],
    df_filtered['label'],
    test_size=0.2,
    random_state=42
)

X_train_encoded = tokenizer(list(X_train), padding=True, truncation=True, return_tensors='pt', max_length=128)
X_test_encoded = tokenizer(list(X_test), padding=True, truncation=True, return_tensors='pt', max_length=128)
y_train_tensor = torch.tensor(list(y_train))
y_test_tensor = torch.tensor(list(y_test))

In [20]:
# Create DataLoader
train_dataset = TensorDataset(X_train_encoded.input_ids, X_train_encoded.attention_mask, y_train_tensor)
test_dataset = TensorDataset(X_test_encoded.input_ids, X_test_encoded.attention_mask, y_test_tensor)

In [21]:
# Use GPU (T4) if available, otherwise use CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Move the model and datasets to the GPU
model.to(device)
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=32)

In [25]:
# Define the BERT fine-tuning parameters
num_epochs = 3
optimizer = AdamW(model.parameters(), lr=2e-5, eps=1e-8)
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=len(train_dataloader) * num_epochs
)
loss_fn = nn.CrossEntropyLoss()

In [26]:
# Training loop
for epoch in range(num_epochs):
    model.train()
    total_loss = 0.0
    for batch in train_dataloader:
        input_ids, attention_mask, labels = batch
        input_ids, attention_mask, labels = (
            input_ids.to(device),
            attention_mask.to(device),
            labels.to(device)
        )
        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)  # Optional gradient clipping
        optimizer.step()
        scheduler.step()
        total_loss += loss.item()

    avg_train_loss = total_loss / len(train_dataloader)
    print(f"Epoch {epoch + 1}/{num_epochs}, Average Training Loss: {avg_train_loss:.4f}")

Epoch 1/3, Average Training Loss: 0.2559
Epoch 2/3, Average Training Loss: 0.0967
Epoch 3/3, Average Training Loss: 0.0437


In [27]:
# Evaluation
model.eval()
all_preds = []
with torch.no_grad():
    for batch in test_dataloader:
        input_ids, attention_mask, labels = batch
        input_ids, attention_mask = input_ids.to(device), attention_mask.to(device)
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        preds = np.argmax(logits.to("cpu").numpy(), axis=1)
        all_preds.extend(preds)

# Calculate accuracy and generate classification report
accuracy = accuracy_score(list(y_test), all_preds)
print(f"Accuracy: {accuracy:.2f}")

classification_rep = classification_report(list(y_test), all_preds)
print(classification_rep)

Accuracy: 0.97
              precision    recall  f1-score   support

           0       0.96      0.98      0.97      2067
           1       0.98      0.96      0.97      2010

    accuracy                           0.97      4077
   macro avg       0.97      0.97      0.97      4077
weighted avg       0.97      0.97      0.97      4077



In [34]:
# Define the output directory within your Colab workspace
output_dir = "/content/drive/MyDrive/my_project/models"  # Replace with your desired path

# Save the model to the specified directory
model.save_pretrained(output_dir)

# Save the tokenizer as well
tokenizer.save_pretrained(output_dir)

('/content/drive/MyDrive/my_project/models/tokenizer_config.json',
 '/content/drive/MyDrive/my_project/models/special_tokens_map.json',
 '/content/drive/MyDrive/my_project/models/vocab.txt',
 '/content/drive/MyDrive/my_project/models/added_tokens.json')